In [67]:
import pandas as pd
import requests
import re
import numpy as np

In [ ]:
# 1. open the latest file
try:
    df = pd.read_csv('results.csv')
except FileNotFoundError:
    df = pd.read_excel('FR Extension Products (2024.10.15.).xlsx',sheet_name='Data 1110 (3)')

entries = []
debug=False

documents = df[df['Testing Phase'].isna()]['document_number'].values
for i,ID in enumerate(documents):

    # 1. get document text url
    text_url = requests.get(f'https://www.federalregister.gov/api/v1/documents/{ID}.json').json()['raw_text_url']
    print(f'[{i+1}/{len(documents)}] {text_url}')
    
    # 2. retrieve document text
    text = requests.get(text_url).content.decode('utf-8')

    try:       
        # 3. parse text
        name_patent_company = None
        def match(text):
            yield re.search(r'for\s+([()\u2122<>,\-A-z\s0-9]+?)\s+\(U\.S\.\s+Patent\s+No.*?\s+(.*?)\)\s+from\s+(.*?),?\s+and', text,re.DOTALL)
            yield re.search(r'of\s+([<>,\-A-z\s0-9]+?)\s+\(U\.S\.\s+Patent\s+No.*?\s+(.*?)\)\s+from\s+(.*?),?\s+and', text,re.DOTALL)
        if ID=='2021-23719':
            product_name = 'SPINRAZA'
            patent_no = 'U.S. Patent Nos. 7,838,657 and 8,110,560 from University of Massachusetts, and U.S. Patent Nos. 8,361,977 and 8,980,853 from Biogen MA Inc., and Cold Spring Harbor Laboratory'
            company = 'University of Massachusetts and Biogen MA Inc., and Cold Spring Harbor Laboratory'
        else:
            for result in match(text):
                name_patent_company = result
                if name_patent_company is not None:
                    break
            product_name = name_patent_company.group(1).replace('\n','')
            patent_no = name_patent_company.group(2).replace('\n','').replace(',','')
            company = name_patent_company.group(3).replace('\n','')
        periods = re.search(r'regulatory\sreview\speriod.*?([\d,]+)\s+days.*?([\d,]+)\s+days.*?([\d,]+)\s+days', text, re.DOTALL)
    
        nda_pma_filing_date = None
        def match(text):
            yield re.search(r'new\s+drug\s+application.*?([A-z]+\s+[0-9]+,\s+[0-9]+)', text, re.DOTALL)
            yield re.search(r'1\..*?([A-z]+\s+[0-9]+,\s+[0-9]+).*?IDE\s+effective\s+date', text, re.DOTALL)
            yield re.search(r'1\..*?FDA\s+has\s+verified.*?([A-z]+\s+[0-9]+,\s+[0-9]+)', text, re.DOTALL)
        for result in match(text):
            nda_pma_filing_date = result
            if nda_pma_filing_date is not None:
                break
        nda_pma_filing_date = nda_pma_filing_date.group(1).replace('\n','')
    
        pat_term_ext = None
        def match(text):
            yield re.search(r'([\d,]+)\s+days\s+of\s+patent\s+term\s+extension', text)
            yield re.search(r'3\..*?applicant\s+seeks\s([\d,]+)\s+days', text, re.DOTALL)
            yield re.search(r'3\..*?applicant\s+seeks\s([\d,]+)\s+years', text, re.DOTALL) # time specified in years NOT days
        if ID=='2019-25821':
            pat_term_ext = '5 days, 1,168 days, or 501 days'
        elif re.search(r'applicant\s+seeks\s+zero\s+days\s+of\s+patent\s+term\s+extension', text) is not None:
            pat_term_ext = '0'            
        else:
            for result in match(text):
                pat_term_ext = result
                if pat_term_ext is not None:
                    break
            pat_term_ext = pat_term_ext.group(1).replace('\n','').replace(',','')
    
        approval_date = None
        def match(text):
            yield re.search(r'date\s+the\s+application\s+was\s+approved.*?([A-z]+\s+[0-9]+,\s+[0-9]+)', text, re.DOTALL)
            yield re.search(r'3\..*?FDA\s+has\s+verified.*?approved.*?([A-z]+\s+[0-9]+,\s+[0-9]+)', text, re.DOTALL)
        for result in match(text):
            approval_date = result
            if approval_date is not None:
                break
        approval_date = approval_date.group(1).replace('\n','')

        match = re.search(r'(NDA|ANDA|BLA|IND|PMA|NADA|PLA)\s+(B?P?[0-9]+-?[0-9]*)', text, re.DOTALL)
        application_no = match.group(1)+' '+match.group(2)
    
        # 4. assemble entry
        entry = {
            "document_number": ID,
            "product_name": product_name,
            "patent_no": patent_no,
            "company": company,
            "reg_review_period": periods.group(1).replace('\n','').replace(',',''),
            "testing_phase": periods.group(2).replace('\n','').replace(',',''),
            "approval_phase": periods.group(3).replace('\n','').replace(',',''),
            "ind_die_filing_date": re.search(r'1\..*?([A-z]+\s+[0-9]+,\s+[0-9]+)', text, re.DOTALL).group(1).replace('\n',''),
            "nda_pma_filing_date": nda_pma_filing_date,
            "approval_date": approval_date,
            "application_no": application_no,
            "pat_term_ext": pat_term_ext,
        }
        
        # 5. append to list
        entries.append(entry)
    except AttributeError as e:
        print(f'Failed to parse {ID}')
        if debug: raise

# 6. update dataframe
for e in entries:
    loc = df.index[df['document_number'] == e['document_number']].tolist()[0]
    df.loc[loc, 'Product Name'] = e['product_name']
    df.loc[loc, 'Patent No.'] = e['patent_no']
    df.loc[loc, 'Company'] = e['company']
    df.loc[loc, 'Reg Rev Period'] = float(e['reg_review_period'])
    df.loc[loc, 'Testing Phase'] = e['testing_phase']
    df.loc[loc, 'Approval Phase'] = float(e['approval_phase'])
    df.loc[loc, 'IND/IDE Filing Date'] = e['ind_die_filing_date']
    df.loc[loc, 'NDA/PMA Filing Date'] = e['nda_pma_filing_date']
    df.loc[loc, 'Approval Date'] = e['approval_date']
    df.loc[loc, 'Application No.'] = e['application_no']
    df.loc[loc, 'Pat Term Ext'] = e['pat_term_ext']

# 8. save file
df.to_csv('results.csv',index=False)